Dataset: https://www.kaggle.com/dhruvildave/spotify-charts?select=charts.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Notes: 



*   70 regions.


### Importing necessary libraries.

In [ ]:
!pip3 install spotipy
import pandas as pd
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

     |████████████████████████████████| 138 kB 43.0 MB/s 
     |████████████████████████████████| 62 kB 889 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Loading data.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ML-Project/data/charts.csv")
df.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Despacito (Featuring Daddy Yankee),1,2017-03-01,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Argentina,top200,SAME_POSITION,365941.0
1,El Amante,2,2017-03-01,Nicky Jam,https://open.spotify.com/track/3umS4y3uQDkqekN...,Argentina,top200,SAME_POSITION,179697.0
2,Reggaetón Lento (Bailemos),3,2017-03-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,SAME_POSITION,169647.0
3,Shape of You,4,2017-03-01,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Argentina,top200,SAME_POSITION,168495.0
4,Chantaje (feat. Maluma),5,2017-03-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,141696.0


In [ ]:
len(pd.unique(df['region']))

70

In [ ]:
len(df)

In [ ]:
len(pd.unique(top200['date'])) # Unique dates Top200.

In [ ]:
top200 = df[df['chart'] == "top200"]
viral50 = df[df['chart'] == 'viral50']

In [ ]:
len(pd.unique(viral50['date'])) # Unique dates Viral50.

In [ ]:
print("Length Top 200:", len(top200))
print("Length Viral 50", len(viral50))

Length Top 200: 19873410
Length Viral 50 5638792


### Cleaning Top200 data

In [ ]:
top200.isnull().sum()

title      7
rank       0
date       0
artist     0
url        0
region     0
chart      0
trend      0
streams    0
dtype: int64

In [ ]:
top200.columns.to_list()

['title',
 'rank',
 'date',
 'artist',
 'url',
 'region',
 'chart',
 'trend',
 'streams']

In [ ]:
top200 = top200[top200['title'].notna()]

In [ ]:
top200.isnull().sum()

title      0
rank       0
date       0
artist     0
region     0
chart      0
trend      0
streams    0
id         0
dtype: int64

### Adding acoustic parameters, popularity, etc to Top200.

In [ ]:
top200 = top200[top200['region'] == "Ireland"]
top200['id'] = top200['url'].apply(lambda x: x.split('/')[4])
top200.drop('url',axis=1,inplace=True)

In [ ]:
len(top200)

352390

In [ ]:
client_id = '7ac5c16c450e4d59bd2e9cb28dd6f81b'
client_secret = '3811b847b45d4b6282ecc6a8a11878ad'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
song_meta={'id':[],'album':[], 'artist':[], 'explicit':[], 'popularity':[]}

songs_already_used = []
for song_id in top200['id']:
    # get song's meta data
     # check whether the song has already been used or not
    if song_id in songs_already_used:
      continue
    else:
      songs_already_used.append(song_id)
    meta = sp.track(song_id)
    song_meta['id'].append(song_id)
    album = meta['album']['name']
    song_meta['album']+=[album]
    # song name
    #song=meta['name']
    #song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

# check the song feature
#features = sp.audio_features(song_meta['id'])
#Direct call throwing an API error ( GET too many )
#-------------------------------------------------------
#features = sp.audio_features(song_meta['id'])
#--------------------------------------------------------
features = []
for song_id in songs_already_used:
  features.append(sp.audio_features(song_id)[0])
# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)


# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000
song_meta_df = pd.DataFrame.from_dict(song_meta)

# combine two dataframe
final_df = song_meta_df.merge(features_df)
top200_df = final_df.merge(top200)
top200_df = top200_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)

top200_df = top200_df.sort_values(["date", "rank"], ascending = (True, True))
top200_df.to_csv('/content/top200.csv')

AttributeError: ignored

In [ ]:
#features = []
#for song_id in songs_already_used:
  #features.append(sp.audio_features(song_id)[0])

# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)


# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000
song_meta_df = pd.DataFrame.from_dict(song_meta)

# combine two dataframe
final_df = song_meta_df.merge(features_df)
top200_df = final_df.merge(top200)
top200_df = top200_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)

top200_df = top200_df.sort_values(["date", "rank"], ascending = (True, True))
top200_df.to_csv('/content/top200.csv')



len(features)

AttributeError: ignored

In [ ]:
top200_df.head()

,id,album,artist,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,title,rank,date,region,chart,trend,streams
0,3umS4y3uQDkqekNjVpiRUs,Fénix,Nicky Jam,False,69,0.683,0.691,8,-5.535,1,0.0432,0.243,0.0,0.1400,0.732,179.910,audio_features,spotify:track:3umS4y3uQDkqekNjVpiRUs,https://api.spotify.com/v1/tracks/3umS4y3uQDkq...,https://api.spotify.com/v1/audio-analysis/3umS...,3.658450,4,El Amante,2,2017-03-01,Argentina,top200,SAME_POSITION,179697.0
1,3AEZUABDXNtecAOSC1qTfo,Primera Cita,CNCO,False,73,0.761,0.838,4,-3.073,0,0.0502,0.400,0.0,0.1760,0.710,93.974,audio_features,spotify:track:3AEZUABDXNtecAOSC1qTfo,https://api.spotify.com/v1/tracks/3AEZUABDXNte...,https://api.spotify.com/v1/audio-analysis/3AEZ...,3.709333,4,Reggaetón Lento (Bailemos),3,2017-03-01,Argentina,top200,SAME_POSITION,169647.0
2,7qiZfU4dY1lWllzX7mPBI3,÷ (Deluxe),Ed Sheeran,False,86,0.825,0.652,1,-3.183,0,0.0802,0.581,0.0,0.0931,0.931,95.977,audio_features,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,3.895217,4,Shape of You,4,2017-03-01,Argentina,top200,SAME_POSITION,168495.0


In [ ]:
top200_df.columns.to_list()

['id',
 'album',
 'artist',
 'explicit',
 'popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'title',
 'rank',
 'date',
 'region',
 'chart',
 'trend',
 'streams']

### Cleaning Viral50 Data.

In [ ]:
viral50.isnull().sum()

title            4
rank             0
date             0
artist          18
url              0
region           0
chart            0
trend            0
streams    5638792
dtype: int64

In [ ]:
viral50 = viral50[viral50['region'] == "Ireland"]
viral50 = viral50[viral50['title'].notna()]
viral50 = viral50[viral50['artist'].notna()]

viral50.isnull().sum()

title     0
rank      0
date      0
artist    0
url       0
region    0
chart     0
trend     0
id        0
dtype: int64

In [ ]:
#viral50.drop("streams",axis=1,inplace=True)
viral50.isnull().sum()

title     0
rank      0
date      0
artist    0
url       0
region    0
chart     0
trend     0
id        0
dtype: int64

In [ ]:
(len(pd.unique(viral50['id'])))

7776

In [ ]:
viral50['id'] = viral50['url'].apply(lambda x: x.split('/')[4])
song_meta={'id':[],'album':[], 'artist':[], 'explicit':[], 'popularity':[]}

count = 0
#stop = 2
songs_already_used = []
for song_id in viral50['id']:
    # get song's meta data
    count += 1
    # check whether the song has already been used or not
    if song_id in songs_already_used:
      continue
    else:
      songs_already_used.append(song_id)
    meta = sp.track(song_id)
    song_meta['id'].append(song_id)
    album = meta['album']['name']
    song_meta['album']+=[album]
    # song name
    #song=meta['name']
    #song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)
    #just for testing purposes
    #if (count % stop) == 0:
      #print(count)
      #break
# check the song feature
#Direct call throwing an API error ( GET too many )
#-------------------------------------------------------
#features = sp.audio_features(song_meta['id'])
#--------------------------------------------------------
features = []
for song_id in songs_already_used:
  features.append(sp.audio_features(song_id)[0])

# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)
#features_df.to_csv('/content/features.csv')

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000
song_meta_df = pd.DataFrame.from_dict(song_meta)
#song_meta_df.to_csv('/content/song_meta.csv')

# combine two dataframe
#final_df = song_meta_df.merge(features_df)
#viral50_df = final_df.merge(viral50)
#viral50_df = viral50_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)
#viral50_df = viral50_df.sort_values(["date", "rank"], ascending = (True, True))
#viral50_df.to_csv('/content/viral.csv')

In [ ]:
final_df = song_meta_df.merge(features_df , on = "id" , how = "outer")
viral50_df = final_df.merge(viral50, on = "id", how = "outer")
viral50_df = viral50_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)
viral50_df = viral50_df.sort_values(["date", "rank"], ascending = (True, True))
viral50_df.to_csv('/content/viral.csv')

In [ ]:
print(len(features))
viral50_df.isnull().sum()

7116


id                  0
album               0
artist_x            0
explicit            0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
duration_ms         0
time_signature      0
title               0
rank                0
date                0
artist_y            0
url                 0
region              0
chart               0
trend               0
dtype: int64

In [ ]:
#top200['id'] = top200['url'].apply(lambda x: x.split('/')[4])
song_meta={'id':[],'album':[], 'artist':[], 'explicit':[], 'popularity':[]}
songs_already_used = []
for song_id in top200['id']:
    # get song's meta data
    count += 1
    # check whether the song has already been used or not
    if song_id in songs_already_used:
      continue
    
    meta = sp.track(song_id)
    if not bool(meta):
      continue
    songs_already_used.append(song_id)
    song_meta['id'].append(song_id)
    album = meta['album']['name']
    song_meta['album']+=[album]
    # song name
    #song=meta['name']
    #song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)
    #just for testing purposes
    #if (count % stop) == 0:
      #print(count)
      #break
# check the song feature
#Direct call throwing an API error ( GET too many )
#-------------------------------------------------------
#features = sp.audio_features(song_meta['id'])
#--------------------------------------------------------






KeyboardInterrupt: ignored

In [ ]:

features = []
for song_id in songs_already_used:
  ftr = sp.audio_features(song_id)[0]
  if bool(ftr):
    features.append(ftr)

# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)
#features_df.to_csv('/content/features.csv')

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000
song_meta_df = pd.DataFrame.from_dict(song_meta)
#song_meta_df.to_csv('/content/song_meta.csv')

final_df = song_meta_df.merge(features_df , on = "id" , how = "outer")
top200_df = final_df.merge(top200, on = "id", how = "outer")
top200_df = top200_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)
top200_df = top200_df.sort_values(["date", "rank"], ascending = (True, True))
top200_df.to_csv('/content/top.csv')

In [ ]:
len(top200)
top200_df.isnull().sum()

id                  0
album               0
artist_x            0
explicit            0
popularity          0
danceability        1
energy              1
key                 1
loudness            1
mode                1
speechiness         1
acousticness        1
instrumentalness    1
liveness            1
valence             1
tempo               1
type                1
duration_ms         1
time_signature      1
title               0
rank                0
date                0
artist_y            0
region              0
chart               0
trend               0
streams             0
dtype: int64

In [ ]:
null = top200_df[top200_df["energy"].isnull()]
b = null["title"].values[0]
print(b)
check = top200_df[top200_df['title'] == b]
print(check)
#ftr = sp.audio_features("3uMIfxV8q5tPTWzJh8s6Mk")
#print(ftr)


One Love Manchester - Live from One Love Manchester
                            id  ... streams
102087  3uMIfxV8q5tPTWzJh8s6Mk  ...  3336.0

[1 rows x 27 columns]
